## question 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
def generate_population(size, x1_bounds, x2_bounds,x3_bounds):
    population = []

    for _ in range(size):
        individual = {
            "x1": np.random.uniform(*x1_bounds),
            "x2": np.random.uniform(*x2_bounds),
            "x3": np.random.uniform(*x3_bounds)           
        }
        population.append(individual)

    return population

In [8]:
def objective_fn(individual):
    x1= individual['x1']
    x2= individual['x2']
    x3= individual['x3']    
    return math.sqrt(x1)+math.sqrt(x2)+math.sqrt(x3)

In [9]:
def choose_individual(population, minimize=False):
    costs = np.array(list(map(objective_fn, population)))

    min_cost = np.min(costs)
    if min_cost < 0:
        costs += -min_cost
    
    if minimize:
        costs = costs.max() - costs
    
    costs_sum = np.sum(costs)
    return np.random.choice(population, 1, p=[c/costs_sum for c in costs])[0]

def cross_over(a, b,c):
    return {
        "x1": np.mean([a['x1'], b['x1'],c['x1']]),
        "x2": np.mean([a['x2'], b['x2'],c['x2']]),
        "x3": np.mean([a['x3'], b['x3'],c['x3']]),
    }

def mutate(individual, x_bounds, y_bounds):
    return {
        "x": np.clip(individual["x"] + np.random.uniform(-0.05, 0.05), x_bounds[0], x_bounds[1]),
        "y": np.clip(individual["y"] + np.random.uniform(-0.05, 0.05), y_bounds[0], y_bounds[1])
    }

def gen_next_pop(population, x_bounds, y_bounds):
    gen = []

    for _ in range(len(population)):
        a = choose_individual(population, minimize=True)
        b = choose_individual(population, minimize=True)

        individual = cross_over(a, b)
        individual = mutate(individual, x_bounds, y_bounds)
        gen.append(individual)

    return gen

def plot(x_bounds, y_bounds, mins, figsize=(15, 20)):
    fig = plt.figure(figsize=figsize)
    ax = fig.gca(projection="3d")

    x = np.linspace(*x_bounds, 100)
    y = np.linspace(*y_bounds, 100)
    x, y = np.meshgrid(x, y)
    z = x**2 + y**2

    x1 = np.array([data['x'] for data in mins])
    y1 = np.array([data['y'] for data in mins])
    z1 = x1**2 + y1**2

    ax.plot_surface(x, y, z, alpha=0.5, cmap="viridis")
    ax.scatter3D(x1, y1, z1, color="red")

In [10]:
x_bounds = [-4, 4]
y_bounds = [-4, 4]
gens = 100
population = generate_population(30, x_bounds, y_bounds)
mins = []

for i in range(gens):
    print(f"🧬 generation {i}")

    population = gen_next_pop(population, x_bounds, y_bounds)

    costs = np.array([objective_fn(individual) for individual in population])
    best_idx = np.argmin(costs)
    mins.append(population[best_idx])
    
    for individual in population:
        print(individual, objective_fn(individual))

print(f"\nbest: {population[best_idx]}: {costs[best_idx]}")

TypeError: generate_population() missing 1 required positional argument: 'x3_bounds'

In [22]:

import numpy as np

# Particle Swarm Optimization
def PSO(problem, MaxIter = 100, PopSize = 100, c1 = 1.4962, c2 = 1.4962, w = 0.7298, wdamp = 1.0):

    # Empty Particle Template
    empty_particle = {
        'position': None, 
        'velocity': None, 
        'cost': None, 
        'best_position': None, 
        'best_cost': None, 
    }

    # Extract Problem Info
    CostFunction = problem['CostFunction']
    VarMin = problem['VarMin']
    VarMax = problem['VarMax']
    nVar = problem['nVar']

    # Initialize Global Best
    gbest = {'position': None, 'cost': np.inf}

    # Create Initial Population
    pop = []
    for i in range(0, PopSize):
        pop.append(empty_particle.copy())
        pop[i]['position'] = np.random.uniform(VarMin, VarMax, nVar)
        pop[i]['velocity'] = np.zeros(nVar)
        pop[i]['cost'] = CostFunction(pop[i]['position'])
        pop[i]['best_position'] = pop[i]['position'].copy()
        pop[i]['best_cost'] = pop[i]['cost']
        
        if pop[i]['best_cost'] < gbest['cost']:
            gbest['position'] = pop[i]['best_position'].copy()
            gbest['cost'] = pop[i]['best_cost']
    
    # PSO Loop
    for it in range(0, MaxIter):
        for i in range(0, PopSize):
            
            pop[i]['velocity'] = w*pop[i]['velocity'] \
                + c1*np.random.rand(nVar)*(pop[i]['best_position'] - pop[i]['position']) \
                + c2*np.random.rand(nVar)*(gbest['position'] - pop[i]['position'])

            pop[i]['position'] += pop[i]['velocity']
            pop[i]['position'] = np.maximum(pop[i]['position'], VarMin)
            pop[i]['position'] = np.minimum(pop[i]['position'], VarMax)

            pop[i]['cost'] = CostFunction(pop[i]['position'])
            
            if pop[i]['cost'] < pop[i]['best_cost']:
                pop[i]['best_position'] = pop[i]['position'].copy()
                pop[i]['best_cost'] = pop[i]['cost']

                if pop[i]['best_cost'] < gbest['cost']:
                    gbest['position'] = pop[i]['best_position'].copy()
                    gbest['cost'] = pop[i]['best_cost']

        w *= wdamp
        print('Iteration {}: Best Cost = {}'.format(it, gbest['cost']))

    return gbest, pop


In [24]:

# A Sample Cost Function
def Sphere(x):
    return 400/x + 2*np.pi*(x**2)

# Define Optimization Problem
problem = {
        'CostFunction': Sphere, 
        'nVar': 10, 
        'VarMin': 0,   
        'VarMax': 5,  
    }

# Running PSO
print('Running PSO ...')
gbest, pop = PSO(problem, MaxIter = 200, PopSize = 50, c1 = 1.5, c2 = 2, w = 1, wdamp = 0.995)

# Final Result
print('Global Best:')
print(gbest)
print()


Running PSO ...


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()